# Stock Market Price Prediction using LSTMs

In [ ]:
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override()

In [ ]:
df = pdr.get_data_yahoo('SALASAR.NS')

In [ ]:
df.to_csv('SALASAR.csv')

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('SALASAR.csv')

In [ ]:
df

In [ ]:
df2 = df.reset_index()['Close']

In [ ]:
df2

In [ ]:
df2.shape

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(df2)
plt.xlabel('Days')
plt.ylabel('Close Price')
plt.title('Salasar Techno')
plt.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
mm = MinMaxScaler(feature_range=(0,1))

In [ ]:
df2 = mm.fit_transform(np.array(df2).reshape(-1,1)) 

In [ ]:
df2

In [ ]:
training_size = int(len(df2)*0.7)
test_size = len(df2) - training_size

In [ ]:
training_size

In [ ]:
test_size

In [ ]:
train_data,test_data = df2[:training_size,:],df2[training_size:len(df2),:1]

In [ ]:
train_data

In [ ]:
test_data

In [ ]:
def create_dataset(dataset, time_stemp=1):
    xdata,ydata = [], []
    for i in range(len(dataset)-time_stemp-1):
        a = dataset[i:(i+time_stemp),0]
        xdata.append(a)
        ydata.append(dataset[i+time_stemp,0])
    return np.array(xdata), np.array(ydata)

In [ ]:
batch = 200

In [ ]:
time_stemp = batch
x_train, y_train = create_dataset(train_data, time_stemp)
x_test, y_test = create_dataset(test_data, time_stemp)

In [ ]:
x_train, y_train

In [ ]:
len(x_train)

In [ ]:
len(x_test)

In [ ]:
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense

In [ ]:
model = Sequential()

model.add(LSTM(50, return_sequences=True, input_shape=(200,1)))
model.add(LSTM(50, return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1))

In [ ]:
model.compile(optimizer='Adam', loss='mean_squared_error')

In [ ]:
model.summary()

In [ ]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=100,batch_size=64)

In [ ]:
train_predict = model.predict(x_train)
test_predict = model.predict(x_test)

In [ ]:
train_predict = mm.inverse_transform(train_predict)
test_predict = mm.inverse_transform(test_predict)

In [ ]:
import math
from sklearn.metrics import mean_squared_error

In [ ]:
math.sqrt(mean_squared_error(y_train,train_predict))

In [ ]:
math.sqrt(mean_squared_error(y_test,test_predict))

In [ ]:
look_back = batch
trainpredictplot = np.empty_like(df2)
trainpredictplot[:,:] = np.nan
trainpredictplot[look_back:len(train_predict)+look_back,:] = train_predict

testpredictplot = np.empty_like(df2)
testpredictplot[:,:] = np.nan
testpredictplot[len(train_predict)+(look_back*2)+1:len(df2)-1,:] = test_predict

plt.plot(mm.inverse_transform(df2))
plt.plot(trainpredictplot)
plt.plot(testpredictplot)
plt.show()

In [ ]:
len(test_data)

In [ ]:
x_input = test_data[290:].reshape(-1,1)

In [ ]:
x_input.shape

In [ ]:
temp_input = list(x_input)

In [ ]:
temp_input = [array[0] for array in temp_input]

In [ ]:
temp_input

In [ ]:
from numpy import array

lst_output=[]
n_steps=207
i=0
while(i<30):
    
    if(len(temp_input)>200):
        x_input=np.array(temp_input[1:])
        print("{} day input {}".format(i,x_input))
        x_input=x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))
        yhat = model.predict(x_input, verbose=0)
        print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        lst_output.extend(yhat.tolist())
        i=i+1
    else:
        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input, verbose=0)
        print(yhat[0])
        temp_input.extend(yhat[0].tolist())
        print(len(temp_input))
        lst_output.extend(yhat.tolist())
        i=i+1
    

print(lst_output)

In [ ]:
day_new=np.arange(1,201)
day_pred=np.arange(201,231)

In [ ]:
num1 = len(df2) - batch

In [ ]:
num1

In [ ]:
plt.plot(day_new,mm.inverse_transform(df2[num1:]))
plt.plot(day_pred,mm.inverse_transform(lst_output))

In [ ]:
df3=df2.tolist()
df3.extend(lst_output)
plt.plot(df3[num1 + 100:])

In [ ]:
df3=mm.inverse_transform(df3).tolist()

In [ ]:
plt.plot(df3)

In [ ]:
plt.plot(df3[len(df2):])

In [ ]:
from keras.models import load_model

In [ ]:
# model.save('salasarmodel')